In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute,mean,std


In [ ]:
data = pd.read_csv('master_df.csv')
data.head()


In [ ]:
cell_name = list(data["cell_num"].unique())
print(cell_name)


In [ ]:
data.set_index("cell_num",inplace=True)

In [ ]:
group_data=data.groupby(["cell_num"])

In [ ]:
X = []
Y = []
for i in cell_name:
    if(i not in ['b3c7', 'b3c16', 'b3c37', 'b3c44']):
        temp_data = group_data.get_group(i).head(75)
        temp_data = temp_data.drop(["cycle_life"], axis=1)
        x1 = []
        for j in range(len(temp_data)):
            x1.append(list(temp_data.iloc[j]))
        X.append(x1)
        Y.append(int(data.loc[i, "cycle_life"].head(1)))


print(X[0][0:2])
print(Y[0:2])


In [ ]:
print(len(X),len(Y))

In [ ]:
X = np.array(X)
X.shape


In [ ]:
X=np.reshape(X,(len(X),75*7))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
# Scale x and y (two scale objects)
X = sc_x.fit_transform(X)
Y = np.array(Y)
Y = sc_y.fit_transform(Y.reshape(-1, 1))
X_train, X_test, Y_train, Y_test,= train_test_split(X,Y,test_size=0.3,random_state=0)

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
import sklearn.metrics
from math import sqrt


model = ElasticNet(alpha=0.001432, l1_ratio=0.605,
                   fit_intercept=False, random_state=0, tol=0.0001)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
pred = Y_pred
pred = sc_y.inverse_transform(pred.reshape(-1, 1))
Y_test = sc_y.inverse_transform(Y_test.reshape(-1, 1))
# print(Y_test[0:2])
# print(pred[0:2])
mean_absolute_percentage_error = sklearn.metrics.mean_absolute_percentage_error(
    Y_test, pred)
mean_absolute_error = sklearn.metrics.mean_absolute_error(Y_test, pred)
mean_sqaured_error = sklearn.metrics.mean_squared_error(
    Y_test, pred, squared=True)
root_mean_sqaured_error = sklearn.metrics.mean_squared_error(
    Y_test, pred, squared=False)

print("MAPE:", mean_absolute_percentage_error*100, "%")
print("MAE:", mean_absolute_error)
print("MSE:", mean_sqaured_error)
print("RMSE:", root_mean_sqaured_error)




In [ ]:
import matplotlib.pyplot as plt

def predvsactual():
    fig = plt.figure(figsize=(12, 5))
    markerline1, stemlines1, baseline1 = plt.stem(Y_test)
    markerline1.set_markerfacecolor('red')
    markerline, stemlines, baseline = plt.stem(pred)
    markerline.set_markerfacecolor('blue')
    plt.legend(['Actual_cycle_life', 'Predicted_cycle_life'])
    plt.xlabel('Cells')
    plt.ylabel('Cycle Life')
    plt.show()


predvsactual()
